# 7월 25일

- IAM 서비스 권한 경계
    - AWS의 모든 서비스는 IAM에 권한 제어를 받는다.
    - 권한 부여 없이 모든 권한을 가지는 것이 root user(email)

    - 버전 관리는 활성화시 비용이 들어간다.

    - cloudfront를 사용하면 엣지 로케이션을 통해 S3 오리진 서버에서 직접 데이터를 제공하는 대신 캐싱된 데이터를 제공하여 대역폭 비용 절감 효과를 기대할 수 있다.

    - 단순하게 빠른 로드 분산이 필요하다 --> NLB
    - 지능적인 라우팅과 다양한 프로토콜 지원이 가능하여 HTTP/HTTPS 기반 복잡한 요청 처리가 필요한 경우(트래픽 분석, 분기) --> ALB

# 8. 데이터베잊스 서버 준비하기
- 데이터베이스 서버란?
- RDS
- 파라미터 그룹/옵션 그룹 생성하기
- 서브넷 그룹 생성하기
- 데이터베이스 생성하기
- DynamoDB


## 데이터베이스 서버란?
- DBMS
    - 사용자가 관리도구로 SQL문을 이용해서 질의를 하면, 데이터베이스 서버에서 질의에 해당하는 데이터를 조작한다.

- 관계형 DB(RDBMS)
    - 테이블 스키마 형태로 데이터 저장
    - DBMS로 관리(SQL 등)
    

- 비관계형 데이터베이스(NoSQL)
    - 키 밸류 스토어 데이터베이스
    - 문서 기반 데이터베이스
    - 그래프 데이터베이스

- RDS
    - Amazone RDS 기능 - 관계형 데이터베이스의 관리형 서비스
    



- 잘 모르겠다? 
    - 공부해보기(숙제)
    - DB
    - DBMS
    - RDB
    - 간단 SQL: create, insert, select, update, delete


### RDS
- 관계형 데이터베이스의 관리형 서비스
    - 클라우드에서 데이터베이스를 사용할 수 있다.

- AWS의 기본 지원 데이터베이스: Amazone Aurora
- DBMS 솔루션 선택하여 AWS 환경에서 서비스할 수 있도록 도와준다.

- 온프레미스 환경에서는 처음부터 끝까지 직접 설정하여 관리해야 한다.
- EC2 환경에서는 OS 설치까지만 사용하고 나머지는 관리해야 한다.
- RDS/Aurora 환경에서는 App Optimization을 제외하면 AWS에서 관리해둔다.

- 경우에 따라서는 EC2까지만 쓰는 비관리형이 아닌 RDS와 같은 관리형 서비스를 사용하면 관리에 대한 오버로드를 줄여 편리하지만, 사용하지 못하는 경우도 있다. 커널 파라미터에 대한 요구사항이 존재할 시 RDS는 사용할 수 없다. --> EC2를 사용하여 파라미터 조정

- Multi-AZ
    - secondary DB를 만드는 것처럼, RDS도 다중 AZ 배포
    - 마스터 DB에 장애가 났을 때 세컨더리 DB가 자동으로 승격하여 작업을 진행하는 구성
    - 프라이머리 인스턴스 & 예비 복제본을 다중 AZ 배포: 장애 조치를 통한 고가용성
    - 간단하게 사용하겠다는 체크만 하면 사용할 수 있다.

- 읽기 전용 복제본
    - 프라이머리 인스턴스에서 복제된 읽기 전용 인스턴스
    - 읽기가 많은 경우 읽기 전용 복제본으로 읽기 부하를 분산
    - 한 곳에 많은 로드가 집중되는 것을 지양하는 것이 권장사항 --> 읽기 전용 복제본을 통해 로드 분산 가능

- DB Instance는 EC2와 같다고 생각하면 된다. 
- DB 배포 서브넷 지정 필요 --> NACL, 보안 그룹 설정 가능
    - 퍼블릭에 배포 가능하지만, 권장 사항은 불필요한 노출을 최소화 하는 것이기에 프라이빗으로 배포하는 것이 바람직하다.
    - 배포 범위: 서브넷 그룹 선택

- RDS 베이터베이스 백업
    - 자동 백업(보존기간 0~35일): 지정한 일 수 만큼의 스냅샷 자동 유지
        - 0은 자동 백업 비활성화
        - 첫 번째 전체 스냅샷 이후 후속 스냅샷은 증분식(변경분만 저장)
        - 오랜 기간 잡으면 비용이 증가하여 바람직하지 않는다. 
        - RDS 삭제 시 같이 삭제된다.
    - 수동 백업: 수동 스냅샷 생성 --> 35일 이상 백업 보존, RDS 삭제에 영향을 받지 않음
    - 특정 시점 복원(PITR, Point in Time Recovery): 특정 시간을 지정하여 복원(최소 5분전, 보전 기간 내)

### RDS - 생성
- 파라미터 그룹
    - 주로 데이터베이스 엔진 고유 설정, DB 튜닝 설정
- 옵션 그룹
    - RDS 고유 설정, AWS 데이터베이스 모니터링 관련 설정
- 서브넷 그룹 생성
    - DB 인스턴스를 여러 가용 영역에 분산 배치하기 위한 서브넷 모임, 가용성 및 성능을 높일 수 있음

## 데이터베이스 생성하기
### 데이터베이스 생성
1. AWS 로그인
    - 리전 선택 후 관리 콘솔 접속
    - RDS 대시보드 접속

2. DB 인스턴스 생성
    - DB 생성
    - 엔진 선택
    - 템플릿 선택
    - DB 사양 지정
    - 상세 설정

3. RDS 접속
    - EC2에서 RDS로 접속
    - DB 클라이언트(관리 도구)에서 접속

4. 데이터 입력
    - 기존 데이터 가져오기
    - 신규 데이터의 입력, 삭제, 변경하기

### 실습14 - RDS 생성 연결
- Myvpc에 퍼블릭 서브넷1에 RDS 생성
    - 애플리케이션이 없어 퍼블릭에 배포. 앞단에는 웹,애플리케이션이 배치되어 RDS는 프라이빗에 배포하는 것이 바람직하다.

- 보안 그룹: sgdb
    - 인바운드: 3306(mysql/aurora), anywhereipv4 선택

- DB 서브넷 그룹 생성


- VPC 서비스에서 MyVPC의 DNS 호스트 이름 활성화 체크
- DB인스턴스 생성
    - MySQL 선택
    - 엔진 8.0.42
    - 프리티어
    - 이름 및 암호 설정
    - 인스턴스 db.t3.micro
    - 추가 스토리지 구성의 자동 조정 활성화 체크 해제
    - 연결에서 컴퓨팅 리소스 터치 X
    - 퍼블릭 액세스 예로 변경
    - 추가 구성 필수. 초기 데이터베이스 설정(초기 데이터베이스 이름)
    - 백업 보존 기간 1일

- MySQL Workbench 설치
    - 커넥션에서 호스트 네임에 DB인스턴스의 엔트포인트 주소 복사 및 붙여넣기 후 생성
    - create database db001;
    - use db001;
    - CREATE TABLE tbl01
    - (
    - empno INT NOT NULL PRIMARY KEY,
    - ename VARCHAR(10));
    - select * from tbl01;
    - insert into tbl01 values (1,"Emma");
    - select * from tbl01

- 실습 후 DB 인스턴스 삭제


## DynamoDB
- 키 값 데이터 유형의 비관계형(NoSQL) 데이터베디스
- 단순 데이터 구조: 복잡한 검색보다 고속 데이터 검색
- 스키마 유연성이 높음
- 완전 관리형 서비스, 서버리스.
    - 서버의 생성과 관리가 필요 없음. AWS에서 관리
    - 테이블만 생성하여 사용하면 된다.
- 단순 연산만 하는 데이터베이스에 유리
    - 장바구니나 순위 등 단순 데이터들을 담는, 빠르게 바뀌는 데이터들을 다룰 때 적합하다.
- 조인 등의 복잡한 연산이 필요한 경우에는 관계형 데이터가 유리


- 기능 및 특징
    - 일관된 성능: 00ms 미만의 성능
    - 무제한 처리량: 사이즈를 정하지 않음. 사용한 만큼 비용 지불
    - 서버관리 불필요: 테이블과 기본 키(파티션키, 정렬키-optional) 생성 후 즉시 사용 --> **기본 키는 필수적으로 필요**
        - 파티션 키: 온프레미스 환경에서도 DB 로드 분산을 위해 파티셔닝, 샤딩을 한다. 디스크 한 곳에 모든 데이터를 저장하면 쿼리 로드가 집중되기 때문에 이를 여러 저장소로 나누어 저장하여 로드 부하를 줄인다.
        - DynamoDB는 알아서 파티셔닝을 통해 분산 저장을 한다. 고객이 테이블을 만들 때 지정한 파티션 키를 기준으로 자동 분산한다.
        - 한 곳에 테이블의 행이 집중되어 저장되는 핫 파티션이 발생하면, 그 저장소를 재파티션한다. --> 느려질 수 있어 바람직하지 않다. **파티션키의 속성을 최대한 중복되지 않는 속성으로 설정**
        - 관계형 DB는 자동으로 파티셔닝을 하지 않는다.

    - RDS보다 편의성을 높인 완전 관리형 서비스!


    - 사례
        - 수백만 사용자 및 초당 수백만 건의 요청 처리
        - 모바일 게임 순위표, 장바구니 및 고객 프로필

- 비관계형 DB다
    - 각 행의 모든 열 값이 define되어 있어야 하는 관계형 DB와 달리, 모든 행이 동일한 열을 갖지 않을 수 있다!
    - 예: 1,2행은 주소 column을 가지지만, 3행은 없을 수도 있다.

- RDS는 AZ레벨
- DynamoDB는 리전 레벨로, 선택한 리전 안의 가용 영역 안에 테이블 생성하며, 3개 영역에 중복 복제본을 유지하여 내구성을 챙긴다.

- 백업
    - 자동 백업으로 특정 시점 복원 가능
    - 수동 백업 지원

### 실습15 DynamoDB 테이블 생성
- 테이블 생성 (콘솔 > DynamoDB > 테이블) : mytbl01
    - 기본키 설정(파티션키) : id
    - 정렬키: 기본 키의 두 번째 부분으로 사용 가능.
    - 테이블 생성

- 테이블에 데이터 추가(테이블 - 작업 - 항목 생성)
    - 한 행의 속성 설정 가능
    - 추가 항목 생성시 다른 속성 추가해서 생성 해보기(name, addr, phone, class…)

- 테이블 쿼리(DynamoDB > 항목 탐색)
    - 스캔
    - 검색

- 항목 삭제
    - 작업 - 항목 삭제

- 마무리
    - 테이블 삭제
    - cloudwatch 경보 삭제, 온디맨드 백업 생성 X (기본 설정으로 삭제)


### 요약 키워드
- 데이터베이스 유형
- 관리형 서비스 VS 비관리형 서비스
    - OS 설치 및 세팅이 간편한 관리형 서비스
    - 비관리형 서비스는 EC2까지만 사용하여 OS패치부터 DB엔진 설정, 설치 등의 작업을 모두 수동으로 하는 방식이다.
- Amazone RDS
- Amazone DynamoDB




# 17. 부록 A - 그 외 서비스
- Lambda
- 컨테이너 서비스
- 프라이빗 네트워크 옵션

## AWS Lambda
### Lambda
- Lambda : **AWS 서버리스 애플리케이션**
    - 서버리스는 서버는 있지만, 고객의 입장에서 서버의 생성과 관리에 신경 쓸 필요가 없는 것을 의미한다.
    - 애플리케이션 실행을 위한 서버 인프라를 생성, 관리할 필요없이 간편하게 애플리케이션 생성 실행
    - 지원 SDK : Python, Java, .NET, Node.js, Go, Ruby

- Lambda 활용 예시
    - 애플리케이션을 제공하려면, EC2 생성 후 APP을 올려 서비스를 하는 것이 일반적이다.
    - 서버에 대한 모니터링, 앱의 정상 가동 확인, 백업, 중복 인스턴스를 준비하는 등의 과정이 필요한데, 너무 복잡해지기 때문에 단순히 애플리케이션을 실행하기 위해 사용하는 것이 Lambda이다.
    - AWS의 이벤트에 의해 호출 될 수도 있다.
    - 일정 예약, 또는 이벤트 트리거로 인해 특정 이벤트가 있을 때 실행하도록 설정할 수 있다.

    - S3에 업로드 하는 이벤트가 있을 때 미리 만들어 둔 람다를 실행하는 것
        1. 파일을 저장
        2. 파일 저장이 트리거가 되어 함수 호출
        3. 썸네일 생성(람다에 작성되어 있는 썸네일 생성 함수)
    
    - Lambda가 이벤트를 읽는 서비스
        - Kinesis
        - DynamoDB
        - SQS
    - Lambda 함수를 동기적으로 호출하는 서비스
        - ELB
        - ALB
        - Congntio
        - Lex
        - Alexa
        - API Gateway
        - CloudFront
        - Kinesis Data Firehose
    - Lambda 함수를 비동기적으로 호출하는 서비스
        - S3
        - SNS
        - SES
        - CloudFormation
        - CloudWatch Logs
        - CloudWatch Events
        - CloudCommit
        - Config


### Lambda의 특징
- 권한
    - 실행 역할, 직책

- 서버를 신경 쓰진 않지만, 서버 자체는 필요하다.
    - CPU는 지정하지 않는다.
    - 서버에 필요한 메모리 지정 필요: 128~10240M
    - 시간 제한: 1초~15분 (기본 3초, 이 시간안에 완료되지 않으면 롤백)
    - 실행 시에만 요금 부과
    - 메모리와도 관련이 있지만, 실행 중이면 요금이 부과되는 EC2와 달리 실행 시에만 요금 부과를 하기 때문에 경우에 따라 선택한다.
        - 앱이 실행 요청의 빈도 수가 많거나, 많이 무거우면 EC2가 비용 효율적일 수 있다.
        - 무조건 Lambda가 옳은 것이 아니다.
        - Lambda는 서버리스가 가장 큰 매력! (EC2 생성 필요 X)


### Lambda 실행 역할
- lambda가 dynamoDB를 건드려야 할 때, 서비스가 서비스에 접근하는 것이기 때문에 IAM Role이 필요하다. 권한을 위임받아야 작업이 가능하다.

- lambda 코드가 서비스를 건드리지 않는다면, 실행 역할은 필요 없는 것이라고 생각할 수 있다. 그러나, lambda function을 실행했을 때 output을 디버깅 등을 위해 남기기 위해서 **CloudWatch Logs에 기록**하기 때문에 Role이 필요하다! (필수 사항)
- CW를 건드릴 수 있는 정책이 부여되어 있는 Role이 Lambda에 위임된 상태!
- **반드시** 실행 역할을 정해야 한다.


### 실습16 - Lambda 실행
- https://docs.aws.amazon.com/ko_kr/lambda/latest/dg/with-s3-tutorial.html
- 두 개의 S3 버킷 생성
- 소스 버킷에 파일 업로드 시 
- 이벤트 트리거에 의해 lambda 자동으로 실행
- 목적지 버킷에 저장해주는 함수

- Lambda : 썸네일 이미지 생성
    - S3 버킷 생성(원본:mybucket#### 및 대상본:mybucket####-resized)
        - 두 개의 버킷 생성 필요. (원본), (원본)-resized 형식
    - IAM 역할 및 정책 확인 : CloudWatch logs와 S3 접근 권한을 가진 역할
        - LambdaS3Role / LambdaS3Policy
        - create policy에서 정책 생성. Role에 부여할 logs 서비스 권한
        - 정책 이름은 LambdaS3Policy
        - Role 생성: LambdaS3Role, 사용 사례에서 람다에게 위임, 정책 검색 상자에서 방금 만든 LambdaS3Policy 선택
        - 파이썬 선택

    - 함수 배포 패키지 생성 : lambda_function.zip (생성 혹은 템플릿 다운로드)
        - 파이썬 사용. PIL 라이브러리는 기본 지원 라이브러리가 아니기에 같이 묶은 코드 사용.
        - 다운로드 한 파일 사용

    - Lambda 함수 생성(이미지 크기 조정 함수) : 콘솔 > 람다 > 함수>> CreateThumbnail
        - 파이썬 3.9버전 선택
        - 기존 실행 역할 변경 --> 기존 역할 사용 --> LambdaS3Role 선택

    - Lambda 트리거 구성
        - 원본 버킷(mybucket#### )에 이미지 업로드시 함수 호출
        - 람다 콘솔의 함수 페이지를 열고 함수(CreateThumbnail)를 선택
        - 트리거 추가
        - S3
        - 버킷에서 원본 버킷 선택
        - 이벤트 유형에서 모든 객체 생성 이벤트 선택
        - 재귀 호출에서 확인란을 선택하여 입력 및 출력에 동일한 Amazon S3 버킷 사용이 권장되지 않음을 확인 (체크 박스 선택)
        - 추가

    - 함수 테스트 : 원본 버킷에 이미지파일 업로드 후, 대상 버킷 (mybucket#### -resized)에 썸네일이미지 및 싸이즈 확인

- 마무리
    - 람다 함수 삭제
    - S3 버킷들 삭제

## VPC와 다른 네트워크 연결
- AWS VPN(Virtual Private Network)
    - AWS 클라우드와 온프레미스 사이에 보안 연결
    - IPSec을 통해 통신하는 경우 등에 사용하는 것. 
    - 인터넷 기반 터널링(IPSec)
    - 전용 채널 생성
    - 암호화된 터널로 보안을 챙긴다.
    - 상대적으로 느릴 수 있다.
- AWS Direct Connect
    - AWS 클라우드와 온프레미스 사이에 네트워크 전용선 연결
    - 터널링을 사용하지 않는다.
    - 물리적 전용 회선을 이용하여 Public망을 사용하지 않도록 하는 것
    - 콘솔에서 구매하여 사용 가능
    - 인터넷 우회로 보안을 챙긴다.
    - 고속, 저지연
    - VLAN, BGP

### VPN과 Direct Connect 연결 이중화
- 데이터 센터의 게이트웨이(라우터)와 VPC의 가상 프라이빗 게이트웨이를 연결할 때 중복 연결로 장애 시 전환 가능하도록 할 수 있다.
    - 데이터센터의 두 개의 게이트웨이가 Direct, VPN으로 Virtual Private Gateway에 연결할 수 있다.
        - Direct Conect 생성 시점부터 요금이 부과되어 Direct Connect로 이중화 시 이중 비용 부담이 발생한다.
        - 하나는 VPN으로 연결하여 이중화 할 수 있다.

### VPC 피어링
- VPC와 VPC 간에 사설 네트워크 연결
- IGW를 거치지 않고, 피어링 전용 GW 사용

- VPC 피어링 제약
    - 전이적 피어링 지원하지 않음
        - 직접적으로 피어링을 맺지 않은 VPC끼리는 통신이 불가능하다.
    - 중복된 CIDR 주소 연결 불가 
        - 목적지가 같은데 라우팅 테이블이 다르면, 하나는 정상적으로 보내지지 않는다. 두 번째 엔트리는 확인하지 않는다.
        - 피어링 할 수도 있다면 처음부터 다르게 만들 계획이 필요하다.

- VPC 피어링을 남발할 이유는 없다!
    - 인터넷 게이트웨이를 사용하지 않고 VPC를 구성하는 경우가 많아 Public한 통신을 하는 경우가 적다.
    - NAT GW를 사용하면 외부로의 단방향 통신이 가능하다.
    

- VPC가 많아지면 굉장히 복잡해진다.
- 이를 해결하기 위해 나온 것이 Transit 게이트웨이

### Transit 게이트웨이
- 다수의 네트워크들을 허브 앤 스포크 형식으로 사설 연결
- 다수의 VPC에 대해서 VPC 연결을 중앙에서 관리한다.
- VPC와 온프레미스 환경까지 관리한다.